In [9]:
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

# define filepath to read dataset
data_path = './dataset/'

# define column names
columns = ['engine_id', 'time_in_cycles'] + \
          ['operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + \
          [f'sensor_measurement_{i}' for i in range(1, 22)]

train_df = pd.read_csv((data_path+'train_FD001.txt'), sep=r'\s+', header=None, names=columns)
test_df = pd.read_csv((data_path+'test_FD001.txt'), sep=r'\s+', header=None, names=columns)
rul_test_df = pd.read_csv((data_path+'RUL_FD001.txt'), sep=r'\s+', header=None, names=['RUL'])

# Max cycle per engine (각 엔진의 마지막 cycle 을 고장 cycle 로 간주)
max_cycle = train_df.groupby('engine_id')['time_in_cycles'].max()

# Caculate RUL (최대 사이클 - 해당 칼럼 사이클)
train_df = train_df.merge(max_cycle, on='engine_id', suffixes=('', '_max'))
train_df['RUL'] = train_df['time_in_cycles_max'] - train_df['time_in_cycles']
train_df.drop('time_in_cycles_max', axis=1, inplace=True)

# 센서 값이 일정한 (변동이 없는) 컬럼을 제거
sensor_columns = [f'sensor_measurement_{i}' for i in range(1, 22)]
constant_sensors = train_df[sensor_columns].std(axis=0) == 0
train_df.drop(columns=constant_sensors.index[constant_sensors], axis=1, inplace=True)
test_df.drop(columns=constant_sensors.index[constant_sensors], axis=1, inplace=True)

# Define the window size for moving average
window_size = 3  # You can adjust this value

# Function to apply moving average to sensor columns
def apply_moving_average(df):
    sensor_columns = [col for col in df.columns if 'sensor_measurement' in col]
    
    # Group by engine_id and apply moving average
    df_grouped = df.groupby('engine_id')[sensor_columns]
    
    # Apply rolling mean
    df[sensor_columns] = df_grouped.rolling(window=window_size, min_periods=1).mean().reset_index(level=0, drop=True)
    
    return df

# Apply moving average to train_df and test_df
train_df = apply_moving_average(train_df)
test_df = apply_moving_average(test_df)

# Min-Max Scaler
scaler = MinMaxScaler()
sensor_columns = [col for col in train_df.columns if 'sensor_measurement' in col]
train_df[sensor_columns] = scaler.fit_transform(train_df[sensor_columns])
test_df[sensor_columns] = scaler.transform(test_df[sensor_columns])

# 시퀀스 길이 설정
sequence_length = 50  # 50

def create_sequences(data, sequence_length):
    """주어진 데이터에서 시퀀스를 만드는 함수"""
    sequences = []
    targets = []
    
    engines = data['engine_id'].unique()
    
    for engine_id in engines:
        engine_data = data[data['engine_id'] == engine_id]
        engine_values = engine_data.drop(columns=['engine_id', 'RUL']).values
        rul_values = engine_data['RUL'].values
        
        for i in range(len(engine_values) - sequence_length + 1):
            sequences.append(engine_values[i: i + sequence_length])
            targets.append(rul_values[i + sequence_length - 1])  # 시퀀스의 마지막 값이 목표 RUL
            
    return np.array(sequences), np.array(targets)

# Train data에서 시퀀스와 타겟 생성
train_sequences, train_targets = create_sequences(train_df, sequence_length)

# 실제 RUL 값은 numpy 배열로 변환
true_rul = rul_test_df.values.squeeze()  # RUL_FD001.csv에서 실제 RUL 값 (예: rul_test_df = pd.read_csv('RUL_FD001.csv'))

def create_test_sequences(data, sequence_length):
    """테스트 데이터를 위해 마지막 sequence_length만큼 시퀀스를 만드는 함수"""
    sequences = []
    
    engines = data['engine_id'].unique()
    
    for engine_id in engines:
        engine_data = data[data['engine_id'] == engine_id]
        engine_values = engine_data.drop(columns=['engine_id']).values
        
        # 시퀀스 길이가 sequence_length보다 짧으면 앞에 0으로 패딩
        if len(engine_values) < sequence_length:
            padding = np.zeros((sequence_length - len(engine_values), engine_values.shape[1]))
            engine_values = np.vstack((padding, engine_values))
        
        sequences.append(engine_values[-sequence_length:])  # 마지막 sequence_length 만큼 사용
    
    return np.array(sequences)

# Test data에서 시퀀스 생성
test_sequences = create_test_sequences(test_df, sequence_length)

# 데이터를 텐서로 변환
train_sequences_tensor = torch.tensor(train_sequences, dtype=torch.float32)
train_targets_tensor = torch.tensor(train_targets, dtype=torch.float32)
test_sequences_tensor = torch.tensor(test_sequences, dtype=torch.float32)
test_targets_tensor = torch.tensor(true_rul, dtype=torch.float32)

# 랜덤 시드 고정
def set_seed(seed=369):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # RUL을 예측하기 위한 출력 레이어
        self.best_state_dict = None

    def forward(self, x):
        # LSTM에 입력: (batch_size, sequence_length, input_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # hidden state 초기화
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # cell state 초기화
        
        # LSTM의 출력: (batch_size, sequence_length, hidden_size), (hn, cn)
        out, _ = self.lstm(x, (h0, c0))
        
        # 최종 타임스텝의 출력만 사용
        out = out[:, -1, :]  # (batch_size, hidden_size)
        
        # Fully connected layer를 통해 RUL 예측
        out = self.fc(out)
        return out

    # 최적의 가중치 저장하는 함수
    def save_best_state_dict(self):
        self.best_state_dict = self.state_dict()
        return self.best_state_dict

    # 최적의 가중치를 반환하는 함수
    def get_best_state_dict(self):
        return self.best_state_dict

# 스코어 함수 선언 asymmetric_scoring 함수는 조기 예측과 늦은 예측에 대해 서로 다른 가중치를 적용하여 스코어를 계산한다. (조기 예측 가중치 a1, 늦은 예측 가중치 a2)
def asymmetric_scoring(y_true, y_pred, a1=10, a2=13):
    """
    비대칭 스코어링 함수
    y_true: 실제 RUL 값 (numpy array)
    y_pred: 예측된 RUL 값 (numpy array)
    a1: 조기 예측에 대한 가중치
    a2: 늦은 예측에 대한 가중치
    """
    errors = y_pred - y_true
    scores = np.where(errors < 0, np.exp(-errors / a1) - 1, np.exp(errors / a2) - 1)
    return np.sum(scores)

# evaluate_algorithm 함수는 여러 UUT 에 대해 총 스코어를 계산한다.
def evaluate_algorithm(y_true_all, y_pred_all, a1=10, a2=13):
    """
    알고리즘 평가 함수
    y_true_all: 실제 RUL 값 리스트 (각 UUT별 numpy array)
    y_pred_all: 예측된 RUL 값 리스트 (각 UUT별 numpy array)
    a1: 조기 예측에 대한 가중치
    a2: 늦은 예측에 대한 가중치
    """
    total_score = 0

    for y_true, y_pred in zip(y_true_all, y_pred_all):
        score = asymmetric_scoring(y_true, y_pred, a1, a2)
        total_score += score
    
    return total_score

In [10]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score

input_size = train_sequences.shape[2]  # 센서/피처 수
hidden_size = 50  # LSTM hidden state 크기 (높을 수로 모델 복잡해짐)
num_layers = 2  # LSTM 레이어 수
learning_rate = 0.0005

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_size, num_layers).to(device)  # 모델을 GPU로 이동

# 랜덤 시드 고정
set_seed(369)

# 손실 함수 및 옵티마이저 정의
criterion = nn.MSELoss()  # RUL 예측이므로 MSELoss 사용
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam Optimizer 사용

# 학습 설정
num_epochs = 200
batch_size = 128

# Early Stopping을 위한 변수 추가
best_val_loss = float('inf')
patience = 6 # 조기 종료를 위한 patience 값 설정 (100 epoch 동안 개선이 없으면 종료)
epochs_no_improve = 0 # 개선이 없었던 epoch 수를 기록하는 변수

# 텐서로 변환된 학습 데이터 (train_sequences_tensor, train_targets_tensor)를 DataLoader로 묶기
train_dataset = TensorDataset(train_sequences_tensor, train_targets_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 검증 데이터셋 크기 증가 (훈련 데이터의 20% 정도 추천)
val_size = int(len(train_sequences_tensor) * 0.2)  # 훈련 데이터의 20%를 검증 데이터로 사용
val_sequences_tensor, val_targets_tensor = train_sequences_tensor[:val_size], train_targets_tensor[:val_size]
train_sequences_tensor, train_targets_tensor = train_sequences_tensor[val_size:], train_targets_tensor[val_size:]

val_dataset = TensorDataset(val_sequences_tensor, val_targets_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 저장을 위한 초기 설정
best_model_path = 'best_model.pth'

# 학습 및 검증 루프 + Early Stopping
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, targets in train_loader:
        sequences = sequences.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 검증 단계
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for sequences, targets in val_loader:
            sequences = sequences.to(device)
            targets = targets.to(device)
            outputs = model(sequences)
            loss = criterion(outputs.squeeze(), targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Val Loss: {avg_val_loss:.4f}")

    # Early Stopping 구현
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        epochs_no_improve = 0
        print("Saved Best Model")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break



# 테스트 데이터 예측
model.load_state_dict(torch.load(best_model_path))
model.eval()  # 평가 모드로 전환 (드롭아웃 등을 비활성화)
with torch.no_grad():
    test_sequences_tensor = test_sequences_tensor.to(device)  # 테스트 데이터도 GPU로 이동
    predicted_rul = model(test_sequences_tensor)

# 모델이 예측한 RUL 값
predicted_rul = predicted_rul.cpu().numpy().squeeze()  # GPU에서 예측한 값을 numpy로 변환

total_score = evaluate_algorithm(true_rul, predicted_rul)
rmse = np.sqrt(mean_squared_error(true_rul, predicted_rul))
r2 = r2_score(true_rul, predicted_rul)

# 결과 출력
print(f"Test Score: {total_score}")
print(f'Test RMSE: {rmse:.2f}')
print(f'Test R² Score: {r2:.4f}')

Epoch [1/200], Loss: 9884.8270, Val Loss: 8922.8880
Saved Best Model
Epoch [2/200], Loss: 9047.0278, Val Loss: 8361.5311
Saved Best Model
Epoch [3/200], Loss: 8538.6224, Val Loss: 7887.4917
Saved Best Model
Epoch [4/200], Loss: 8086.4916, Val Loss: 7458.1187
Saved Best Model
Epoch [5/200], Loss: 7673.3734, Val Loss: 7061.5686
Saved Best Model
Epoch [6/200], Loss: 7292.3280, Val Loss: 6695.0668
Saved Best Model
Epoch [7/200], Loss: 6937.7239, Val Loss: 6353.6251
Saved Best Model
Epoch [8/200], Loss: 6608.7850, Val Loss: 6035.7595
Saved Best Model
Epoch [9/200], Loss: 6302.1702, Val Loss: 5742.6881
Saved Best Model
Epoch [10/200], Loss: 6017.6382, Val Loss: 5467.8375
Saved Best Model
Epoch [11/200], Loss: 5752.5210, Val Loss: 5213.4697
Saved Best Model
Epoch [12/200], Loss: 5507.4771, Val Loss: 4978.0144
Saved Best Model
Epoch [13/200], Loss: 5280.9361, Val Loss: 4760.1729
Saved Best Model
Epoch [14/200], Loss: 5070.6263, Val Loss: 4558.7296
Saved Best Model
Epoch [15/200], Loss: 4877.37

/tmp/ipykernel_10910/4132973108.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))
